In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies 
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Preprocess

In [3]:
# Read in files

file_path = 'Data/Cleaned_Data/'

In [4]:
# Read in Health Ranking Data
df = pd.read_csv(f"../../{file_path}County_Health_All_Factors.csv")
print(df.shape)
df.head()

(3142, 10)


,Unnamed: 0,FIPS,State,County,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank
0,1,1001,Alabama,Autauga,8,5,15,14,5,50
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62
2,3,1005,Alabama,Barbour,14,48,53,30,61,32
3,4,1007,Alabama,Bibb,47,24,38,16,38,31
4,5,1009,Alabama,Blount,36,14,10,41,15,53


In [5]:
# Read in the Merged Data
df2 = pd.read_csv(f"../../{file_path}Merged_Demographic_Data.csv")
print(df2.shape)
df2.head()

(3033, 25)


,County,State,FIPS,Lyme_Disease_Incidence_Reported,Population,%<18_Yrs_Old,%65_Yrs_Old_and_over,Income($),African_American(Count),African_American(%),...,Native_Hawaiian/Other_Pacific_Islander(%),Hispanic(Count),Hispanic(%),Non-Hispanic_White_(Count),Non-Hispanic_White(%),Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count)
0,Autauga,Alabama,1001,1.0,55504,23.94,15.12,$58343.00,10687,19.25,...,0.10,1586,2.86,41336,74.47,51.34,22921.0,42.00,76.33,815.0
1,Baldwin,Alabama,1003,0.0,212628,21.85,19.95,$56607.00,19037,8.95,...,0.07,9675,4.55,176582,83.05,51.45,77060.0,42.28,78.60,2827.0
2,Barbour,Alabama,1005,1.0,25270,20.76,18.82,$32490.00,12115,47.94,...,0.19,1063,4.21,11613,45.96,47.23,18613.0,67.79,75.78,451.0
3,Bibb,Alabama,1007,0.0,22668,20.61,16.02,$45795.00,4864,21.46,...,0.11,598,2.64,16842,74.30,46.45,15663.0,68.35,73.93,445.0
4,Blount,Alabama,1009,0.0,58013,23.35,17.84,$48253.00,847,1.46,...,0.12,5549,9.57,50439,86.94,50.69,51562.0,89.95,74.60,1050.0


In [6]:
df_merged = df.merge(df2, how='left', on='FIPS')
print(df_merged.shape)
df_merged.head(10)

(3142, 34)


,Unnamed: 0,FIPS,State_x,County_x,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,...,Native_Hawaiian/Other_Pacific_Islander(%),Hispanic(Count),Hispanic(%),Non-Hispanic_White_(Count),Non-Hispanic_White(%),Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count)
0,1,1001,Alabama,Autauga,8,5,15,14,5,50,...,0.10,1586.0,2.86,41336.0,74.47,51.34,22921.0,42.00,76.33,815.0
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62,...,0.07,9675.0,4.55,176582.0,83.05,51.45,77060.0,42.28,78.60,2827.0
2,3,1005,Alabama,Barbour,14,48,53,30,61,32,...,0.19,1063.0,4.21,11613.0,45.96,47.23,18613.0,67.79,75.78,451.0
3,4,1007,Alabama,Bibb,47,24,38,16,38,31,...,0.11,598.0,2.64,16842.0,74.30,46.45,15663.0,68.35,73.93,445.0
4,5,1009,Alabama,Blount,36,14,10,41,15,53,...,0.12,5549.0,9.57,50439.0,86.94,50.69,51562.0,89.95,74.60,1050.0
5,6,1011,Alabama,Bullock,53,58,65,62,64,24,...,0.78,850.0,8.25,2196.0,21.30,45.53,5607.0,51.37,73.12,205.0
6,7,1013,Alabama,Butler,63,56,56,52,55,8,...,0.05,285.0,1.44,10229.0,51.60,53.43,14921.0,71.23,73.51,393.0
7,8,1015,Alabama,Calhoun,55,10,28,34,27,41,...,0.11,4302.0,3.75,82961.0,72.31,51.93,39955.0,33.70,73.10,2333.0
8,9,1017,Alabama,Chambers,42,53,57,25,37,46,...,0.04,822.0,2.44,18710.0,55.50,52.13,16816.0,49.15,74.12,691.0
9,10,1019,Alabama,Cherokee,43,8,12,54,19,43,...,0.03,425.0,1.64,23707.0,91.69,50.32,22282.0,85.74,74.35,575.0


In [7]:
df_merged = df_merged.dropna()

In [8]:
# Check for null values
for col in df_merged:
    print(f'{col} has {df_merged[col].isnull().sum()} nulls')

Unnamed: 0 has 0 nulls
FIPS has 0 nulls
State_x has 0 nulls
County_x has 0 nulls
Lifespan_Rank has 0 nulls
Life_Quality_Rank has 0 nulls
Health_Behaviors_Rank has 0 nulls
Clinical_Care_Rank has 0 nulls
Social_Economic_Factors_Rank has 0 nulls
Physical_Environment_Rank has 0 nulls
County_y has 0 nulls
State_y has 0 nulls
Lyme_Disease_Incidence_Reported has 0 nulls
Population has 0 nulls
%<18_Yrs_Old has 0 nulls
%65_Yrs_Old_and_over has 0 nulls
Income($) has 0 nulls
African_American(Count) has 0 nulls
African_American(%) has 0 nulls
American_Indian/Alaskan_Native(Count) has 0 nulls
American_Indian/Alaskan_Native(%) has 0 nulls
Asian(Count) has 0 nulls
Asian(%) has 0 nulls
Native_Hawaiian/Other_Pacific_Islander(Count) has 0 nulls
Native_Hawaiian/Other_Pacific_Islander(%) has 0 nulls
Hispanic(Count) has 0 nulls
Hispanic(%) has 0 nulls
Non-Hispanic_White_(Count) has 0 nulls
Non-Hispanic_White(%) has 0 nulls
Female(%) has 0 nulls
Rural(Count) has 0 nulls
Rural(%) has 0 nulls
Life_Expectancy 

In [9]:
df_merged = df_merged.drop(columns=['Unnamed: 0','County_y','State_y','State_x','County_x'])

In [10]:
df_merged['Income($)'] = df_merged['Income($)'].str.replace('$', '')
df_merged['Income($)'] = df_merged['Income($)'].astype('float') / 100

In [11]:
df_merged = df_merged.astype({'Lifespan_Rank': 'int32', 'Life_Quality_Rank': 'int32','Health_Behaviors_Rank':'int32',
'Clinical_Care_Rank':'int32','Social_Economic_Factors_Rank':'int32','Physical_Environment_Rank':'int32'})

In [12]:
df_merged.dtypes

FIPS                                               int64
Lifespan_Rank                                      int32
Life_Quality_Rank                                  int32
Health_Behaviors_Rank                              int32
Clinical_Care_Rank                                 int32
Social_Economic_Factors_Rank                       int32
Physical_Environment_Rank                          int32
Lyme_Disease_Incidence_Reported                  float64
Population                                       float64
%<18_Yrs_Old                                     float64
%65_Yrs_Old_and_over                             float64
Income($)                                        float64
African_American(Count)                          float64
African_American(%)                              float64
American_Indian/Alaskan_Native(Count)            float64
American_Indian/Alaskan_Native(%)                float64
Asian(Count)                                     float64
Asian(%)                       

In [14]:
df3 = pd.read_csv(f"../../{file_path}Ticks_Cases_Lat_Long.csv")
df3.head()

,Unnamed: 0,FIPS,County,State,Lat,Long,Ticks_With_Lyme,Cases2019
0,0,1001.0,Autauga County,Alabama,32.5349,-86.6427,0,1
1,1,1003.0,Baldwin County,Alabama,30.7275,-87.7226,0,0
2,2,1005.0,Barbour County,Alabama,31.8696,-85.3932,0,1
3,3,1007.0,Bibb County,Alabama,32.9986,-87.1265,0,0
4,4,1009.0,Blount County,Alabama,33.9809,-86.5674,0,0


In [49]:
mldf = df_merged.merge(df3, on='FIPS')
mldf = mldf.drop(columns=['Cases2019','Unnamed: 0'])
print(mldf.shape)
mldf.head()

(3013, 34)


,FIPS,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Lyme_Disease_Incidence_Reported,Population,%<18_Yrs_Old,...,Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count),County,State,Lat,Long,Ticks_With_Lyme
0,1001,8,5,15,14,5,50,1.0,55504.0,23.94,...,51.34,22921.0,42.00,76.33,815.0,Autauga County,Alabama,32.5349,-86.6427,0
1,1003,3,4,3,4,3,62,0.0,212628.0,21.85,...,51.45,77060.0,42.28,78.60,2827.0,Baldwin County,Alabama,30.7275,-87.7226,0
2,1005,14,48,53,30,61,32,1.0,25270.0,20.76,...,47.23,18613.0,67.79,75.78,451.0,Barbour County,Alabama,31.8696,-85.3932,0
3,1007,47,24,38,16,38,31,0.0,22668.0,20.61,...,46.45,15663.0,68.35,73.93,445.0,Bibb County,Alabama,32.9986,-87.1265,0
4,1009,36,14,10,41,15,53,0.0,58013.0,23.35,...,50.69,51562.0,89.95,74.60,1050.0,Blount County,Alabama,33.9809,-86.5674,0


In [ ]:
# Check for null values
for col in mldf:
    print(f'{col} has {mldf[col].isnull().sum()} nulls')

In [17]:
mldf.dtypes

FIPS                                               int64
Lifespan_Rank                                      int32
Life_Quality_Rank                                  int32
Health_Behaviors_Rank                              int32
Clinical_Care_Rank                                 int32
Social_Economic_Factors_Rank                       int32
Physical_Environment_Rank                          int32
Lyme_Disease_Incidence_Reported                  float64
Population                                       float64
%<18_Yrs_Old                                     float64
%65_Yrs_Old_and_over                             float64
Income($)                                        float64
African_American(Count)                          float64
African_American(%)                              float64
American_Indian/Alaskan_Native(Count)            float64
American_Indian/Alaskan_Native(%)                float64
Asian(Count)                                     float64
Asian(%)                       

In [40]:
Lyme_col = mldf[['Lyme_Disease_Incidence_Reported']]

# Establish the spending bins and group names.
lyme_bins = [-1 ,0, 1, 2, 3, 500]
group_names = ["0", "1", "2", "3","4+"]

# Categorize spending based on the bins.
Lyme_col['Lyme_Cat'] = pd.cut(Lyme_col['Lyme_Disease_Incidence_Reported'], lyme_bins, labels=group_names)
Lyme_col.sample(15)

,Lyme_Disease_Incidence_Reported,Lyme_Cat
1883,2.0,2
2572,0.0,0
613,2.0,2
2502,0.0,0
2059,1.0,1
1796,57.0,4+
2645,0.0,0
1123,1.0,1
954,0.0,0
1053,0.0,0


In [43]:
# Lyme disease incident reported bin key
ldir_key_df = pd.DataFrame({
    "bin_sizes":['0','1','2','3','4+'],
    "bin_ids": [0,1,2,3,4]
})
ldir_key_df.head()

,bin_sizes,bin_ids
0,0,0
1,1,1
2,2,2
3,3,3
4,4+,4


In [48]:
def map_fxn (x):
    if x >4:
        return 4
    return x

In [50]:
# Map value to mldf to match the key df
mldf['Lyme_Disease_Incidence_Reported']= mldf['Lyme_Disease_Incidence_Reported'].map(map_fxn)
mldf.head()

,FIPS,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Lyme_Disease_Incidence_Reported,Population,%<18_Yrs_Old,...,Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count),County,State,Lat,Long,Ticks_With_Lyme
0,1001,8,5,15,14,5,50,1.0,55504.0,23.94,...,51.34,22921.0,42.00,76.33,815.0,Autauga County,Alabama,32.5349,-86.6427,0
1,1003,3,4,3,4,3,62,0.0,212628.0,21.85,...,51.45,77060.0,42.28,78.60,2827.0,Baldwin County,Alabama,30.7275,-87.7226,0
2,1005,14,48,53,30,61,32,1.0,25270.0,20.76,...,47.23,18613.0,67.79,75.78,451.0,Barbour County,Alabama,31.8696,-85.3932,0
3,1007,47,24,38,16,38,31,0.0,22668.0,20.61,...,46.45,15663.0,68.35,73.93,445.0,Bibb County,Alabama,32.9986,-87.1265,0
4,1009,36,14,10,41,15,53,0.0,58013.0,23.35,...,50.69,51562.0,89.95,74.60,1050.0,Blount County,Alabama,33.9809,-86.5674,0


In [52]:
mldf['Lyme_Disease_Incidence_Reported'].value_counts()

0.0    1878
4.0     593
1.0     314
2.0     147
3.0      81
Name: Lyme_Disease_Incidence_Reported, dtype: int64

In [ ]:
# # Export Merged DF as csv
# mldf.to_csv(f"../../{file_path}/ML_Demographic_LymeCase_HealthRank.csv", index=False)

## Split the Data into Training and Testing

In [53]:
# Create our features
X = pd.get_dummies(mldf.drop(columns='Lyme_Disease_Incidence_Reported', axis=1))

# Create our target
y = mldf['Lyme_Disease_Incidence_Reported']
y.value_counts()

0.0    1878
4.0     593
1.0     314
2.0     147
3.0      81
Name: Lyme_Disease_Incidence_Reported, dtype: int64

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape



(2259, 1863)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [55]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0.0: 1397, 3.0: 1397, 4.0: 1397, 1.0: 1397, 2.0: 1397})

In [56]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

In [57]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5212201591511937


In [58]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3,4
0,277,17,20,29,138
1,28,1,6,5,35
2,14,0,4,2,18
3,2,0,1,3,11
4,23,1,7,4,108


In [59]:
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.81      0.58      0.75      0.67      0.66      0.43       481
        1.0       0.05      0.01      0.97      0.02      0.11      0.01        75
        2.0       0.11      0.11      0.95      0.11      0.32      0.09        38
        3.0       0.07      0.18      0.95      0.10      0.41      0.15        17
        4.0       0.35      0.76      0.67      0.48      0.71      0.51       143

avg / total       0.59      0.52      0.77      0.53      0.59      0.38       754



In [75]:
# Different model:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Create a random forest classifier model
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit the model 
brf.fit(X_resampled, y_resampled)

# Make predictions using the testing data
predictions = brf.predict(X_test)

In [77]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.8840948582001215


### SMOTE Oversampling

In [60]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [61]:
# Train the Logistic Regression model using the resampled data
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

In [62]:
# Calculated the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.5265251989389921


In [63]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3,4
0,299,11,48,43,80
1,29,3,13,5,25
2,17,0,8,2,11
3,2,0,3,4,8
4,30,4,20,6,83


In [64]:
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.79      0.62      0.71      0.70      0.67      0.44       481
        1.0       0.17      0.04      0.98      0.06      0.20      0.04        75
        2.0       0.09      0.21      0.88      0.12      0.43      0.17        38
        3.0       0.07      0.24      0.92      0.10      0.47      0.20        17
        4.0       0.40      0.58      0.80      0.47      0.68      0.45       143

avg / total       0.60      0.53      0.77      0.55      0.61      0.38       754



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [65]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0.0: 64, 1.0: 64, 2.0: 64, 3.0: 64, 4.0: 64})

In [66]:
# Train the Logistic Regression model using the resampled data
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

In [67]:
# Calculated the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.5411140583554377


In [68]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3,4
0,309,8,25,48,91
1,30,0,6,8,31
2,16,0,4,3,15
3,2,0,1,4,10
4,33,1,3,15,91


In [69]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.79      0.64      0.70      0.71      0.67      0.45       481
        1.0       0.00      0.00      0.99      0.00      0.00      0.00        75
        2.0       0.10      0.11      0.95      0.10      0.32      0.09        38
        3.0       0.05      0.24      0.90      0.08      0.46      0.20        17
        4.0       0.38      0.64      0.76      0.48      0.70      0.48       143

avg / total       0.58      0.54      0.76      0.55      0.59      0.39       754



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [70]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0.0: 847, 1.0: 1214, 2.0: 1364, 3.0: 1507, 4.0: 1244})

In [71]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = classifier.predict(X_test)

In [72]:
# Calculated the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.5411140583554377


In [73]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3,4
0,309,8,25,48,91
1,30,0,6,8,31
2,16,0,4,3,15
3,2,0,1,4,10
4,33,1,3,15,91


In [74]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.79      0.64      0.70      0.71      0.67      0.45       481
        1.0       0.00      0.00      0.99      0.00      0.00      0.00        75
        2.0       0.10      0.11      0.95      0.10      0.32      0.09        38
        3.0       0.05      0.24      0.90      0.08      0.46      0.20        17
        4.0       0.38      0.64      0.76      0.48      0.70      0.48       143

avg / total       0.58      0.54      0.76      0.55      0.59      0.39       754

